In [1771]:
import math
import numpy as np
import pandas as pd
pinlet=6000.0 #psi
poutlet=3000.0 #psi
pi=4500
x_Tot=1400 #ft
k_x=250*0.001 #md
k_y=250*0.001 #md, this is not given by the question, I assumed since there is no infor about it
h=60 #ft
y_Tot=1400 #ft
mu=1 #cp
fVF=1#RB/STB
phi=0.18 #fraction

#added from C++
alpha_c=5.614583
c_l = 3.5E-06
b_c = 1.127;
b_L = 1;
b_l_0 = b_L;

ansMat=[4500.0 for i in range(51)]
ansMat=np.transpose([ansMat])
ansMat=np.array(ansMat)

#V_b for each grid block shoudl be calculated
#b_c *y_Tot*h*permx / (mu*b_L*dx)
#alpha_c * b_l_0*dt / (v_b*phi*c_l)

# 7x7 matrix is needed

In [1772]:
dx=x_Tot/7.0
dy=y_Tot/7.0
z=h
A_y=dx*h
A_x=dy*h
T=b_c*A_x*k_x/(mu*b_L*dx)
v_b=dy*h*dx
dt=50
TV=alpha_c * b_l_0*50 / (v_b*phi*c_l)
req=0.28*((dx*dx+dy*dy)**0.5)/(2)
jj=abs(-2*3.14159265359*b_c*k_x*h/(mu*b_L*(math.log(req/0.35)-0.5)))
Coe=[[0.0 for i in range(1,52)] for j in range(1,52)]
TMat=[[0.0 for i in range(0,4)] for j in range(0,52)]
Coe=np.array(Coe)
TMat=np.array(TMat)
bound1=np.array([2,3,4,5,6])
bound2=np.array([8,15,22,29,36])
bound3=np.array([44,45,46,47,48])
bound4=np.array([14,21,28,35,42])
FLAG=1

In [1773]:
for x in range(0, 52):
    if(x==1):
        TMat[x,0]=0
        TMat[x,1]=0
        TMat[x,2]=T
        TMat[x,3]=T
        FLAG=1
      
    if(x==7):
        TMat[x,0]=T
        TMat[x,1]=0
        TMat[x,2]=0
        TMat[x,3]=T
        FLAG=1
        
    if(x==43):
        TMat[x,0]=0
        TMat[x,1]=T
        TMat[x,2]=T
        TMat[x,3]=0
        FLAG=1
        
    if(x==49):
        TMat[x,0]=T
        TMat[x,1]=T
        TMat[x,2]=0
        TMat[x,3]=0
        FLAG=1
        
        
    for ii in range(bound1.size):
        if(x==bound1[ii]):
            TMat[x,0]=T
            TMat[x,1]=0
            TMat[x,2]=T
            TMat[x,3]=T
            FLAG=1
            
    for ii in range(bound2.size):
        if(x==bound2[ii]):
            TMat[x,0]=0
            TMat[x,1]=T
            TMat[x,2]=T
            TMat[x,3]=T
            FLAG=1
            
    for ii in range(bound3.size):
        if(x==bound3[ii]):
            TMat[x,0]=T
            TMat[x,1]=T
            TMat[x,2]=T
            TMat[x,3]=0
            FLAG=1
            
    for ii in range(bound4.size):
        if(x==bound4[ii]):
            TMat[x,0]=T
            TMat[x,1]=T
            TMat[x,2]=0
            TMat[x,3]=T
            FLAG=1
        
    if(FLAG==0):
        TMat[x,0]=T
        TMat[x,1]=T
        TMat[x,2]=T
        TMat[x,3]=T
        
    if(x==4 or x==11 or x==18 or x==25):
        TMat[x,2]=0

    if(x==5 or x==12 or x==19 or x==26):
        TMat[x,0]=0
    FLAG=0
    



In [1774]:

Coe[1,1]=(TMat[1,0]+TMat[1,1]+TMat[1,2]+TMat[1,3]+TV)
Coe[1,2]=TMat[1,0]
for i in range(1,50):
    for j in range(1,50):
        if(i==j and i!=7 and i!=43):
            Coe[i,j-1]=TMat[j,0]
            Coe[i,j]=-(TMat[j,0]+TMat[j,1]+TMat[j,2]+TMat[j,3]+TV)
            Coe[i,j+1]=TMat[j,2]
            if(j+7<51):
                Coe[i,j+7]=T
            if(j-7>0):
                Coe[i,j-7]=T
                
        elif(i==j and i==7):
            Coe[i,j-1]=TMat[j,0]
            Coe[i,j]=-(TMat[j,0]+TMat[j,1]+TMat[j,2]+TMat[j,3]+TV+jj)
            Coe[i,j+1]=TMat[j,2]
            Coe[i,j+7]=T
            Coe[i,j+7]=T
        elif(i==j and i==43):
            Coe[i,j-1]=TMat[j,0]
            Coe[i,j]=-(TMat[j,0]+TMat[j,1]+TMat[j,2]+TMat[j,3]+TV-jj)
            Coe[i,j+1]=TMat[j,2]
            Coe[i,j-7]=T
    


In [1775]:
dd=np.delete(Coe,[0] , axis=1)
dd=np.delete(dd,[49] , axis=1)
dd=np.delete(dd,[0] , axis=0)
dd=np.delete(dd,[49] , axis=0)
Qin=[0.0 for i in range(0,49)]
Qin[42]=1
Qin=np.array([Qin])
Qprod=[0.0 for i in range(0,49)]
Qprod[6]=1
Qprod=np.array([Qprod])
QprodT=[0.0 for i in range(0,51)]
QprodT[49]=-1/jj
QinT=[0.0 for i in range(0,51)]
QinT[50]=1/jj
QprodT=np.array([QprodT])
QinT=np.array([QinT])

dd=np.concatenate((dd,Qprod))
dd=np.concatenate((dd,Qin))
dd=np.concatenate((dd,QprodT.T),axis=1)
CoeMatrix=np.concatenate((dd,QinT.T),axis=1)

In [1776]:
df = pd.DataFrame(CoeMatrix)
df.to_csv('CoefficintMat.csv') 

In [1777]:
dd=np.delete(TMat,[0] , axis=0)

In [1778]:

B=[-TV*pi for i in range(51)]
B[6]=B[6]+jj*poutlet
B[42]=B[42]-jj*pinlet
B[49]=3000
B[50]=6000
B=np.array([B])
B=np.transpose(B)


In [1780]:
for k in range(200):
    N = A.shape[0]
    for k in range(N-1):
        for i in range(k+1, N):
            r = -A[i,k] / A[k,k]
            for j in range(k+1, N+1):
                A[i,j] = A[i,j] + r * A[k,j]
            #lines below are not used during back substitution
            for j in range(k+1):
                A[i,j] = 0

    #Back sustitution (in [54])
    A[N-1,N] = A[N-1,-1] / A[N-1, -2]
    for i in range(N-2, -1, -1): #2, 1, 0
        sum = 0
        for j in range(i+1, N): #i+1 to N-1
            sum = sum + A[i,j] * A[j,N]
        A[i,N] = (A[i,N] - sum)/A[i,i]
    A[:,N]
    A
    ans = A[:,N]
    ans = ans[:,np.newaxis]
    ans=np.array(ans)
    if(ans[6]<0):
        ans[6]=0
        
    if(ans[42]>6000):
        ans[42]=6000
    p=[-TV for i in range(51)]
    p=np.array([p])
    p=p.T
    B=p*ans
    B[6]=B[6]+jj*poutlet
    B[42]=B[42]-jj*pinlet
    B[49]=3000
    B[50]=6000
    A = np.hstack((CoeMatrix,B))
    np.linalg.cond(CoeMatrix)
    ansMat=np.concatenate((ansMat,ans),axis=1)
    
df = pd.DataFrame(ansMat)
df.to_csv('ansMat.csv') 

In [1770]:
ans

array([[  5356.47905966],
       [  5332.93639363],
       [  5298.67944317],
       [  5273.40770846],
       [  1223.03891439],
       [   816.90281615],
       [     0.        ],
       [  5395.68047101],
       [  5359.161111  ],
       [  5304.98413076],
       [  5263.25981285],
       [  1622.13862649],
       [  1400.94918569],
       [  1058.695772  ],
       [  5485.83495956],
       [  5417.27055319],
       [  5312.74242464],
       [  5225.03599908],
       [  2234.52449583],
       [  2099.04321116],
       [  1948.03225197],
       [  5656.71445206],
       [  5523.17199149],
       [  5314.94747381],
       [  5109.86037062],
       [  2973.85274303],
       [  2804.22960719],
       [  2678.21513415],
       [  5970.32099866],
       [  5712.41796591],
       [  5321.67289224],
       [  4796.07875839],
       [  3874.74407236],
       [  3456.94774582],
       [  3273.09572961],
       [  6547.81064936],
       [  6039.80044634],
       [  5466.90379717],
       [  48